In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import sagemaker
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sagemaker.amazon.common import write_numpy_to_dense_tensor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker import image_uris
import xgboost as xgb
import io
import shutil

In [ ]:
import boto3
import joblib
from io import BytesIO

file = model_rf
path = 's3://sprint-5-equipe-2'

def write_joblib(file, path):
    if path[:5] == 's3://':
        s3_bucket, s3_key = path.split('/')[2], path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            joblib.dump(file, f)
            f.seek(0)
            boto3.client('s3').upload_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)

    else:
        with open(path, 'wb') as f:
            joblib.dump(file, f)

write_joblib(file, path + '/model.joblib')

In [ ]:
container = get_image_uri(sagemaker_session.boto_region_name, 'xgboost', 'latest')

xgboost = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.m4.xlarge',
                                         output_path='s3://{}/{}/output'.format(bucket, prefix),
                                         sagemaker_session=sagemaker_session)

xgboost.set_hyperparameters(objective='multi:softmax',
                            num_class=3,
                            num_round=100)

data_channels = {
    'train': input_train,
    'validation': input_test
}

xgboost.fit(data_channels)